In [2]:
get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')
from anchor_check import *
from cloth_analysis import *
# import sys
# default_stdout = sys.stdout
# sys.stdout = open('../log/log_prepare_cloth.txt', 'w')

# Create thumbnail for report

In [34]:
anchor_center_dict = pickle.load(open('../data/anchor_center_dict_all.pkl', 'rb'))
# storage = StorageBackend.make_from_config(StorageConfig.make_gcs_config('esper'))
# video_list = sorted(anchor_center_dict)
# random.shuffle(video_list)

In [ ]:
# images = []
for idx, video in enumerate(video_list):
    if len(anchor_center_dict[video]) == 0:
        continue
    fid = [anchor_center_dict[video][0]['fid']]
    video_path = 'tvnews/videos/' + video + '.mp4'
    video_file = RandomReadFile(storage, video_path.encode('ascii'))
    video = Decoder(video_file)
    img = video.retrieve(fid)[0]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    images.append(img)
    if idx > 260:
        break
    print idx

In [ ]:
# grid = stitch_img_grid(images[:192], num_col=16, size=(192, 256), deform=False)
grid = stitch_img_grid(images[:256], num_col=16, size=(144, 256), deform=True)
cv2.imwrite("../data/cloth/plot/motivation.jpg", grid)

In [25]:
path_list = sorted(img_dict)
random.shuffle(path_list)

# Preload all images, pad and resize

In [163]:
# cloth_infer = pickle.load(open('../data/cloth/cloth_all_infer.pkl', 'rb'))
# random.shuffle(cloth_infer)
# trans = {'video': 0, 'anchor_id': 1, 'path': 2, 'bbox': 3, 'attributes': 4}

In [3]:
cloth_dict = pickle.load(open('../data/cloth/newsAnchor_all_data_first.pkl', "rb" ))
random.shuffle(cloth_dict)
trans = {'video': 0, 'anchor_id': 1, 'path': 2, 'detail': 3, 'bbox': 4, 'gender':5, 'attributes': 6, 'feature': 7}

In [ ]:
img_dict = {}
img_dir = '../data/cloth/cloth_all_img'
for cloth in cloth_dict:
    path = cloth[trans['path']]
    bbox = cloth[trans['bbox']]
    img = cv2.imread(os.path.join(img_dir, path))
    if not img is None:
        img_pad = pad_image_height(img, bbox, size=(200, 100))
        img_dict[path] = img_pad

In [ ]:
attribute_id = 1
num_per_class = 500
img_dir = '../data/cloth/cloth_all_img'
images = []
for cloth in cloth_dict:
    attributes = cloth[trans['attributes']]
    attr = int(attributes[attribute_id])
    date, station, show = cloth[trans['detail']]
    if len(images) < num_per_class and attr == 8:
        path = cloth[trans['path']]
#         img_pad = img_dict[path]
        bbox = cloth[trans['bbox']]
        img = cv2.imread(os.path.join(img_dir, path))
        H, W, C = img.shape
        if not img is None and 1. * H / W > 1.9:
            img_pad = pad_image_height(img, bbox, size=(200, 100))
            images.append(img_pad)

random.shuffle(images)
grid = stitch_img_grid(images, 5, size=(200, 100), deform=False)
filename = '../data/cloth/plot/proposal_yellow' + '.jpg'
cv2.imwrite(filename, grid)

# Plot average cloth attributes (for report)

In [ ]:
images_class_grid = []
titles = []

for attribute_id in range(17):
    if attribute_id == 14:
        continue
    
    img_dir = '../data/cloth/cloth_all_img'
    images_class = [[] for i in range(len(attri_dict[attribute_id]))]
    for cloth in cloth_dict:
        attributes = cloth[trans['attributes']]
        if (attribute_id == 12 or attribute_id == 13) and int(attributes[2]) == 0:
            continue
        attr = int(attributes[attribute_id])
        path = cloth[trans['path']]
        img_pad = img_dict[path]
        images_class[attr].append(img_pad)

    for idx, images in enumerate(images_class):
        if len(images) == 0:
            continue
    #     images_class_grid.append(stitch_img_grid(images, 50, size=None))
        # add average
        images_class_grid.append(average_image(images, size=None))

    for idx, t in enumerate(attri_dict_inv[attribute_id].values()):
        if len(images_class[idx]) == 0:
            continue
        titles.append(t)

#     filename = '../data/cloth/plot/average_' + attri_name[attribute_id] + '_report.jpg'
filename = '../data/cloth/plot/average_report.jpg'
multi_plot(images_class_grid, 15, size=(30, 20), titles=titles, output=filename)

# Plot average cloth attributes (separate gender)

In [ ]:
for attribute_id in range(17):
    img_dir = '../data/cloth/cloth_all_img'
    images_class_M = [[] for i in range(len(attri_dict[attribute_id]))]
    images_class_F = [[] for i in range(len(attri_dict[attribute_id]))]
    images_class = [[] for i in range(len(attri_dict[attribute_id]))]
    for cloth in cloth_dict:
        attributes = cloth[trans['attributes']]
        if (attribute_id == 12 or attribute_id == 13) and int(attributes[2]) == 0:
            continue
        attr = int(attributes[attribute_id])
        gender = cloth[trans['gender']]
        path = cloth[trans['path']]
        img_pad = img_dict[path]
        images_class[attr].append(img_pad)
        if gender == 'M':
            images_class_M[attr].append(img_pad)
        elif gender == 'F':
            images_class_F[attr].append(img_pad)

    images_class_grid = []
    for idx, images in enumerate(images_class):
#         if len(images) == 0:
#             continue
    #     images_class_grid.append(stitch_img_grid(images, 50, size=None))
        # add average
        images_class_grid.append(average_image(images, size=None))
        images_class_grid.append(average_image(images_class_M[idx], size=None))
        images_class_grid.append(average_image(images_class_F[idx], size=None))

    titles = []
    for idx, t in enumerate(attri_dict_inv[attribute_id].values()):
#         if len(images_class[idx]) == 0:
#             continue
        titles.append(t + ' All(' + str(len(images_class[idx])) + ')')
        titles.append(t + ' Male(' + str(len(images_class_M[idx])) + ')')
        titles.append(t + ' Female(' + str(len(images_class_F[idx])) + ')')

    filename = '../data/cloth/plot/supply/average_' + attri_name[attribute_id] + '.jpg'
    multi_plot(images_class_grid, 3, size=(16, 10*len(attri_dict[attribute_id])), titles=titles, output=filename)

# check hair length in summer

In [ ]:
attribute_id = 15
num_per_class = 500
img_dir = '../data/cloth/cloth_all_img'
images_class = [[] for i in range(len(attri_dict[attribute_id]))]
for cloth in cloth_dict:
    attributes = cloth[trans['attributes']]
    attr = int(attributes[attribute_id])
    date, station, show = cloth[trans['detail']]
    if len(images_class[attr]) < num_per_class and date[1] >= 7 and date[1] <= 8:
        path = cloth[trans['path']]
        img_pad = img_dict[path]
        images_class[attr].append(img_pad)

images_class_grid = []
for images in images_class:
#         if len(images) == 0:
#             continue
    images_class_grid.append(stitch_img_grid(images, 30, size=None))
    # add average
    images_class_grid.append(average_image(images, size=None))

titles = []
for idx, t in enumerate(attri_dict_inv[attribute_id].values()):
    titles.append(t)
#         if len(images_class[idx]) == 0:
#             continue
    titles.append(t)
filename = '../data/cloth/plot/average_' + attri_name[attribute_id] + '_5000.jpg'
multi_plot(images_class_grid, 5, size=(100, 100), titles=titles, output=filename)

# Plot average male/female host over monthofyear

In [ ]:
images_month_M = [[] for i in range(12)]
images_month_F = [[] for i in range(12)]

num_per_class = 50000
img_dir = '../data/cloth/cloth_all_img'
for cloth in cloth_dict:
    gender = cloth[trans['gender']]
    date, station, show = cloth[trans['detail']]
    
    path = cloth[trans['path']]
#         bbox = cloth[trans['bbox']]
#         img = cv2.imread(os.path.join(img_dir, path))
#         if not img is None:
#             img_pad = pad_image_height(img, bbox, size=(200, 100))
#             images_class[attr].append(img_pad)
    img = img_dict[path]
    if gender == 'M' and len(images_month_M) < num_per_class:
        images_month_M[date[1]-1].append(img)
    elif gender == 'F' and len(images_month_F) < num_per_class:
        images_month_F[date[1]-1].append(img)
        
images_class_grid = []
for images in images_month_M:
    images_class_grid.append(average_image(images, size=None))
for images in images_month_F:
    images_class_grid.append(average_image(images, size=None))    

titles = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
titles += titles

filename = '../data/cloth/plot/report/average_month.jpg'
multi_plot(images_class_grid, 12, size=(24, 8), titles=titles, output=filename)

# grid = stitch_img_grid(images_class_grid, 12, size=None)
# single_plot(grid, size=(16, 16), title='Male/Female averaged by month', output=filename)

# Plot average male/female host over hourofday

In [ ]:
images_hour_M = [[] for i in range(24)]
images_hour_F = [[] for i in range(24)]

def get_hour_bin(date):
    return (date[3] - 5) % 24

num_per_class = 50000
img_dir = '../data/cloth/cloth_all_img'
for cloth in cloth_dict:
    gender = cloth[trans['gender']]
    date, station, show = cloth[trans['detail']]
    path = cloth[trans['path']]
#         bbox = cloth[trans['bbox']]
#         img = cv2.imread(os.path.join(img_dir, path))
#         if not img is None:
#             img_pad = pad_image_height(img, bbox, size=(200, 100))
#             images_class[attr].append(img_pad)
    img = img_dict[path]
    if gender == 'M' and len(images_hour_M) < num_per_class:
        images_hour_M[get_hour_bin(date)].append(img)
    elif gender == 'F' and len(images_hour_F) < num_per_class:
        images_hour_F[get_hour_bin(date)].append(img)
        
images_class_grid = []
for images in images_hour_M:
    images_class_grid.append(average_image(images, size=None))
for images in images_hour_F:
    images_class_grid.append(average_image(images, size=None))    

titles = ['{:02d}'.format(i) for i in range(24)]
titles += titles

filename = '../data/cloth/plot/report/average_hour.jpg'
multi_plot(images_class_grid, 24, size=(32, 6), titles=titles, output=filename)

# grid = stitch_img_grid(images_class_grid, 24, size=None)
# single_plot(grid, size=(32, 16), title='Male/Female averaged by hour', output=filename)

# Plot average male/female host over dayofweek

In [ ]:
images_week_M = [[] for i in range(7)]
images_week_F = [[] for i in range(7)]

num_per_class = 50000
img_dir = '../data/cloth/cloth_all_img'
for cloth in cloth_dict:
    gender = cloth[trans['gender']]
    date, station, show = cloth[trans['detail']]
    path = cloth[trans['path']]
#         bbox = cloth[trans['bbox']]
#         img = cv2.imread(os.path.join(img_dir, path))
#         if not img is None:
#             img_pad = pad_image_height(img, bbox, size=(200, 100))
#             images_class[attr].append(img_pad)
    img = img_dict[path]
    if gender == 'M' and len(images_week_M) < num_per_class:
        images_week_M[get_week_bin(date)].append(img)
    elif gender == 'F' and len(images_week_F) < num_per_class:
        images_week_F[get_week_bin(date)].append(img)
        
images_class_grid = []
for images in images_week_M:
    images_class_grid.append(average_image(images, size=None))
for images in images_week_F:
    images_class_grid.append(average_image(images, size=None))    
# titles = ['Male_M'+str(i) for i in range(1,13) ]
# titles += ['Female_M'+str(i) for i in range(1,13) ]

filename = '../data/cloth/plot/average_dayofweek_5000.jpg'
# multi_plot(images_class_grid, 3, size=(16, 32), titles=titles, output=filename)

grid = stitch_img_grid(images_class_grid, 7, size=None)
single_plot(grid, size=(16, 16), title='Male/Female averaged by dayofweek', output=filename)

# Plot average single host over dayofweek

In [275]:
anchor_center_dict = pickle.load(open('../data/anchor_center_dict_all.pkl', 'rb'))

In [ ]:
anchor_center_show = group_by_show(anchor_center_dict)

for show_name, anchor_center in anchor_center_show.items():
    if len(anchor_center) < 500:
        continue
    # anchor_center = build_anchor_center(anchor_dict_show[show_name])
    real_anchors = cluster_real_anchor(anchor_center_show[show_name])
    for real_anchor_id in range(len(real_anchors)):
        real_anchor = real_anchors[real_anchor_id]

        FACE_SIM_THRESH = 0.9
        img_dir = '../data/cloth/cloth_all_img/'
        images_week = [[] for i in range(7)]
        for cloth in cloth_dict:
            video = cloth[trans['video']]
            anchor_id = cloth[trans['anchor_id']]
            date, station, show = cloth[trans['detail']]
            if show != show_name:
                continue
            ## check face with real anchor
            ## hack for bad anchor detection result
            if anchor_id >= len(anchor_center[video]):
                continue
            center = anchor_center[video][anchor_id] 
            if np.linalg.norm(real_anchor - center['feature']) > FACE_SIM_THRESH:
                continue

            path = cloth[trans['path']]
        #         bbox = cloth[trans['bbox']]
        #         img = cv2.imread(os.path.join(img_dir, path))
        #         if not img is None:
        #             img_pad = pad_image_height(img, bbox, size=(200, 100))
        #             images_class[attr].append(img_pad)
            img = img_dict[path]
            images_week[get_week_bin(date)].append(img)

        images_class_grid = []
        for images in images_week:
            images_class_grid.append(average_image(images, size=None))
#             images_class_grid.append(stitch_img_grid(images, 10, size=None))
        
#         titles = [str(i) for i in range(1,8) ]

        filename = '../data/cloth/plot/average_dayofweek_' + show_name + '_' + str(real_anchor_id) + '.jpg'
#         multi_plot(images_class_grid, 2, size=(50, 150), titles=None, output=filename)

        grid = stitch_img_grid(images_class_grid, 7, size=None)
        single_plot(grid, size=(16, 16), title=show_name + ' averaged by dayofweek', output=filename)

# Plot heatmap of major color&necktie color on single host over dayofweek 

In [ ]:
import seaborn as sns
anchor_center_show = group_by_show(anchor_center_dict)

for show_name, anchor_center in anchor_center_show.items():
    if len(anchor_center) < 500:
        continue
    # anchor_center = build_anchor_center(anchor_dict_show[show_name])
    real_anchors = cluster_real_anchor(anchor_center_show[show_name])
    for real_anchor_id in range(len(real_anchors)):
        real_anchor = real_anchors[real_anchor_id]

        FACE_SIM_THRESH = 0.9
        img_dir = '../data/cloth/cloth_all_img/'
        majorColor_week = [[] for i in range(7)]
        tieColor_week = [[] for i in range(7)]
        heat_matrix_major = np.zeros((len(attri_dict[1]), 7))
        heat_matrix_tie = np.zeros((len(attri_dict[12]), 7))
        for cloth in cloth_dict:
            video = cloth[trans['video']]
            anchor_id = cloth[trans['anchor_id']]
            date, station, show = cloth[trans['detail']]
            if show != show_name:
                continue
            ## check face with real anchor
            ## hack for bad anchor detection result
            if anchor_id >= len(anchor_center[video]):
                continue
            center = anchor_center[video][anchor_id] 
            if np.linalg.norm(real_anchor - center['feature']) > FACE_SIM_THRESH:
                continue
            
            attributes = cloth[trans['attributes']]
            majorColor_week[get_week_bin(date)].append(attributes[1])
            if attributes[2] == 0:
                continue
            tieColor_week[get_week_bin(date)].append(attributes[12])
        
        for i in range(len(attri_dict[1])):
            for j in range(7):
                if len(majorColor_week[j]) == 0:
                    heat_matrix_major[i, j] = 0
                else:
                    heat_matrix_major[i, j] = 1. * majorColor_week[j].count(i) / len(majorColor_week[j])
                if len(tieColor_week[j]) == 0:
                    heat_matrix_tie[i, j] = 0
                else:
                    heat_matrix_tie[i, j] = 1. * tieColor_week[j].count(i) / len(tieColor_week[j])
#         fig = plt.figure()
#         fig.set_figwidth(16)
#         plt.subplot(1,2,1)
        plt.figure()
        y_ticks = attri_dict_inv[1].values()
        x_ticks = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        x_ticks = [x_ticks[i] + ' ({0:d})'.format(len(majorColor_week[i])) for i in range(7)]
        ax = sns.heatmap(heat_matrix_major, vmin=0, vmax=1.0, xticklabels=x_ticks, yticklabels=y_ticks)
        plt.title(show_name + '   Major color')
        filename = '../data/cloth/plot/heatmap_dayofweek_' + show_name + '_' + str(real_anchor_id) + '_majorcolor.png'
        plt.savefig(filename)
        
#         plt.subplot(1,2,2)
        plt.figure()
        y_ticks = attri_dict_inv[12].values()
        x_ticks = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        x_ticks = [x_ticks[i] + ' ({0:d})'.format(len(tieColor_week[i])) for i in range(7)]
        ax = sns.heatmap(heat_matrix_tie, vmin=0, vmax=1.0, xticklabels=x_ticks, yticklabels=y_ticks)
        plt.title(show_name + '   Necktie color')
        filename = '../data/cloth/plot/heatmap_dayofweek_' + show_name + '_' + str(real_anchor_id) + '_necktiecolor.png'
        plt.savefig(filename)
        
#         plt.show()

#         multi_plot(images_class_grid, 2, size=(16, 16), titles=None, output=filename)
#         single_plot(, size=(16, 16), title=show_name + ' averaged by dayofweek', output=filename)

# Plot thumbnail of female hosts in different stations 

In [ ]:
station_name = 'MSNBC'

images = []
img_dir = '../data/cloth/cloth_all_img'
## check cases when anchor are not showing
cnt = 0
for cloth in cloth_dict:
    detail = cloth[trans['detail']]
    if detail[1] != station_name or cloth[trans['gender']] != 'F':
        continue
    img = cv2.imread(os.path.join(img_dir, cloth[trans['path']]))
    H, W, C = img.shape
#     bbox = cloth[trans['bbox']]
#     img_crop = img[bbox['bbox_y2']+20:, :, :]
    img_crop = img[H/2:, :, :]
    images.append(img_crop)
    cnt +=1
    if cnt % 1000 == 0:
        print(cnt)
    if cnt == 6400:
        break

In [ ]:
images_sort = sort_images_by_NN(images)
grid = stitch_img_grid(images_sort, 80, size=(50, 50), deform=True)
cv2.imwrite('../data/cloth/plot/MSNBC_Female_NN.jpg', grid)

# Compare 3 stations

In [60]:
cnn = cv2.imread('../data/cloth/plot/CNN_Female_NN.jpg')
fox = cv2.imread('../data/cloth/plot/FOXNEWS_Female_NN.jpg')
msnbc = cv2.imread('../data/cloth/plot/MSNBC_Female_NN.jpg')

In [ ]:
fig = plt.figure()
fig.set_size_inches(48, 16)
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(cnn, cv2.COLOR_BGR2RGB))
plt.gca().axes.get_yaxis().set_visible(False)
plt.gca().axes.get_xaxis().set_visible(False)
plt.title('CNN')

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(fox, cv2.COLOR_BGR2RGB))
plt.gca().axes.get_yaxis().set_visible(False)
plt.gca().axes.get_xaxis().set_visible(False)
plt.title('FOXNEWS')

plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(msnbc, cv2.COLOR_BGR2RGB))
plt.gca().axes.get_yaxis().set_visible(False)
plt.gca().axes.get_xaxis().set_visible(False)
plt.title('MSNBC')

plt.savefig('../data/cloth/plot/report/anchor_stations_female.png')

# GMM Cluster on feature map 

In [69]:
# PCA first
feature_list = [cloth[trans['feature']] for cloth in cloth_dict]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=200)
feature_pca = pca.fit_transform(np.array(feature_list))

In [ ]:
start = time.time()
NUM_CLUSTER = 77
from sklearn import mixture
gmm = mixture.GaussianMixture(n_components=NUM_CLUSTER, covariance_type='diag')
gmm.fit(feature_pca)
score = gmm.bic(feature_pca)

print(time.time() - start, score)

In [91]:
predicted = gmm.predict(feature_pca)

In [92]:
# collect samples near cluster centers
cluster = []
N = feature_pca.shape[0]
for c in range(NUM_CLUSTER):
    group = [] 
    center = gmm.means_[c]
    for idx in range(N):
        if predicted[idx] == c:
            group.append(idx)
    dist = []
    for idx in group:
        d = np.linalg.norm(center - feature_pca[idx])
        dist.append(d)
    group_sort = np.argsort(dist)
    group = [ group[idx] for idx in group_sort ]
    cluster.append(group)

In [ ]:
import os
NUM_PER_CLUSTER = 20
NUM_AVERAGE = 10000
images = []
img_dir = '../data/cloth/cloth_all_img'
for c, group in enumerate(cluster):
    num_per_cluster = min(NUM_PER_CLUSTER, len(group))
    num_average = min(NUM_AVERAGE, len(group))
    for idx in range(num_per_cluster):
        path = cloth_dict[group[idx]][trans['path']]
        bbox = cloth_dict[group[idx]][trans['bbox']]
        img = cv2.imread(os.path.join(img_dir, path))
        img_pad = pad_image_height(img, bbox)
        images.append(img_pad)
    for i in range(NUM_PER_CLUSTER - num_per_cluster):
        images.append(np.zeros((200, 100, 3)))
        
grid = stitch_img_grid(images, 200, 100, 20, False, True)
cv2.imwrite('../data/cloth/plot/GMM_cluster.jpg', grid)